### Particle Swarm Optimization on Neural Networks
#### References

- J. Kennedy and R. Eberhart, "Particle swarm optimization," Proceedings of ICNN'95 - International Conference on Neural Networks, Perth, WA, Australia, 1995, pp. 1942-1948 vol.4, doi: 10.1109/ICNN.1995.488968.
- M. Kaminski, "Neural Network Training Using Particle Swarm Optimization - a Case Study," 2019 24th International Conference on Methods and Models in Automation and Robotics (MMAR), Miedzyzdroje, Poland, 2019, pp. 115-120, doi: 10.1109/MMAR.2019.8864679.

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,classification_report
import warnings
warnings.filterwarnings("ignore")

In [15]:
import pandas as pd
data = pd.read_csv("data.csv")
print(data.head())

df = data
df = df.drop(['ID','ZIP Code','Personal Loan'],axis=1)
df['Personal Loan'] = data['Personal Loan']


X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

   ID  Age  Experience  Income  ZIP Code  Family  CCAvg  Education  Mortgage  \
0   1   25           1      49     91107       4    1.6          1         0   
1   2   45          19      34     90089       3    1.5          1         0   
2   3   39          15      11     94720       1    1.0          1         0   
3   4   35           9     100     94112       1    2.7          2         0   
4   5   35           8      45     91330       4    1.0          2         0   

   Personal Loan  Securities Account  CD Account  Online  CreditCard  
0              0                   1           0       0           0  
1              0                   1           0       0           0  
2              0                   0           0       0           0  
3              0                   0           0       0           0  
4              0                   0           0       0           1  


In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X= scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(X , y , test_size=0.2, random_state=0)

In [17]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def derivative_sig(x):
    return x * (1 - x)


def forward(
    X, y, input_layer,W1,W2, hidden1, output_layer, iter_, learning_rate
):
    # weights =[]
    # act =[]

    # weights=[W1,W2]

    for i in range(iter_):

        z1 = np.dot(X, W1)
        a1 = sigmoid(z1)
        z2 = np.dot(a1, W2)
        y_pred = sigmoid(z2)

        error = mean_squared_error(y, y_pred)
        delta3 = (y_pred - y) * derivative_sig(y_pred)

        dW2 = np.dot(a1.T, delta3)
        delta2 = np.dot(delta3, W2.T) * derivative_sig(a1)

        dW1 = np.dot(X.T, delta2)

        W1 -= learning_rate * dW1
        W2 -= learning_rate * dW2

    print(f"Error after {iter_} : {error}")

    return W1, W2


def PSO(X, y, W1, W2, iter_, particle_count, var1, var2, var3):
    
    bst_weights = np.zeros(W1.size + W2.size)
    error_ = np.inf
    
    ps_pos = np.random.uniform(-1, 1, size=(particle_count, W1.size + W2.size))
    ps_vel = np.zeros_like(ps_pos)
    ps_bstpos = np.copy(ps_pos)
    ps_error = np.ones(particle_count) * np.inf
    
    for i in range(iter_):
        for j in range(particle_count):

            weights_ = ps_pos[j]
            W1 = weights_[:W1.size].reshape(W1.shape)
            W2 = weights_[W1.size:].reshape(W2.shape)
            
            y_hat = sigmoid(np.dot(sigmoid(np.dot(X, W1)), W2))
            error = np.mean(np.square(y - y_hat))
            
            if error < ps_error[j]:
                ps_bstpos[j] = weights_.copy()
                ps_error[j] = error
            
            if error < error_:
                bst_weights = weights_.copy()
                error_ = error
            
            delta3 = (y_hat - y) * derivative_sig(y_hat)
            dW2 = np.dot(sigmoid(np.dot(X, W1)).T, delta3)
            delta2 = np.dot(delta3, W2.T) * derivative_sig(sigmoid(np.dot(X, W1)))
            dW1 = np.dot(X.T, delta2)
            
            ps_vel[j] = var1 * ps_vel[j] + var2 * np.random.rand() * (ps_bstpos[j] - weights_) + var3 * np.random.rand() * (bst_weights - weights_)
            ps_pos[j] += ps_vel[j]
        
    W1 = bst_weights[:W1.size].reshape(W1.shape)
    W2 = bst_weights[W1.size:].reshape(W2.shape)
    return W1, W2


In [12]:
input_layer = X.shape[1]
hidden1 = 4
output_layer = 1

W1 = np.random.randn(X.shape)
W2 = np.random.randn(y.shape)
learning_rate=0.01
iter_ = 100
particle_count = 50
var1 = 0.5
var2 = 1
var3 = 1
W1 = np.random.randn(input_layer, hidden1)
W2 = np.random.randn(hidden1, output_layer)
w1,w2 =forward(x_train,y_train,W1,W2,input_layer,hidden1,output_layer,iter_,learning_rate)
# print(f"\nInitial Population weight :\n{w1,w2}\n")
# print(f"After ACO :\n{PSO(x_train,y_train,w1,w2,iter_,particle_count,var1,var2,var3)}")
W1,W2 = PSO(x_train,y_train,w1,w2,iter_,particle_count,var1,var2,var3)
w1,w2 =forward(x_test,y_test,W1,W2,input_layer,hidden1,output_layer,iter_,learning_rate)


Classification Report after 1000 :

               precision    recall  f1-score   support

           0       0.50      1.00      0.67         2
           1       1.00      0.60      0.75         5

    accuracy                           0.71         7
   macro avg       0.75      0.80      0.71         7
weighted avg       0.86      0.71      0.73         7

